# ĐỒ ÁN CUỐI KỲ LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU
## THÀNH VIÊN
|MÃ SỐ|HỌ TÊN|
|:----:|:---:|
|19127472|NGUYỄN BÁ MINH|
|19127481|TRẦN HOÀNG NAM|

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

## GIỚI THIỆU ĐỒ ÁN
**Đề tài đồ án**: Tìm hiểu và phân tích những yếu tố trong 10 phút đầu của 1 trận đấu liên minh huyền thoại có ảnh hưởng như thế nào đến kết quả trận đấu
- Dữ liệu các trận đấu được lấy ở [link này](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)
- Dữ liệu có License CC0: Public Domain
- Dữ liệu gồm 1 file duy nhất là data.csv
- Dữ liệu chứa thông tin của khoảng 10.000 trận đấu ở rank kim cương - cao thủ trong khoảng 10 phút đầu game
- Dữ liệu 1 dòng là thông tin của đội xanh và đội đỏ, mỗi đội sẽ có 19 cột chứa thông tin, tổng có 38 cột và 1 cột bluewin là cột kết quả trận đấu, 1 cột id là id của trận đấu, id này có thể dùng để lấy thêm thông tin từ API của riot game, vậy có tất cả 40 cột

### Đọc dữ liệu từ file data.csv và khám phá dữ liệu

In [ ]:
data_df = pd.read_csv('data.csv')
data_df.head()

In [ ]:
data_df.info()

In [ ]:
data_df.shape

#### Dữ liệu có bao nhiêu dòng, cột, có cột nào có dữ liệu thiếu không, có vấn đề gì với các cột không
- Ta thấy rằng có chính xác 40 cột và 9.879 dòng, tức là khoảng 10.000 dòng như mô tả.
- Xem xét sơ các cột, ta thấy rằng không có dữ liệu thiếu, không có vấn đề gì cả.

#### Dữ liệu có bị lặp hay không

In [ ]:
np.any(data_df.duplicated())

### TÌM HIỂU VÀ PHÂN TÍCH

#### NGUYỄN BÁ MINH

Cần tập trung yếu tố trong 10 phút đầu game để giành được chiến thắng

Đầu tiên dễ nhìn thấy nhất là ta sẽ xem thử là lựa chọn màu đội có ảnh hưởng đến tỉ lệ thắng hay không(Trong game thì việc ở đội xanh hay đội đỏ sẽ ảnh hưởng đến việc được cấm chọn trước hay sau)

In [ ]:
blue = data_df['blueWins']
# blue = 1 là thể hiện đội xanh thắng, blue = 0 là đội đỏ thắng
games_count = data_df.shape[0]

In [ ]:
blue_win_rate = (len(data_df[blue == 1])/games_count) * 100
y = [round(blue_win_rate,3),100-round(blue_win_rate,3)]
x = ['Blue', 'Red']


bars = plt.bar(x, height=y, width=.4)

plt.xlabel('Team Color')
plt.ylabel('Win Rate')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x()+0.1, yval + 0.5, f"{yval}%")

plt.show()

Vậy có thể thấy là việc ở team màu nào không ảnh hưởng nhiều đến chiến thắng chung cuộc. 

Tiếp đến ta sẽ đi vào những chỉ số cơ bản nhất của trò chơi đó là số mạng hạ gục, lượng vàng, lượng kinh nghiệm. Ta sẽ tìm hiểu xem sự chênh lệch những yếu tố này trong 10 phút đầu của trận đấu có ảnh hưởng nhiều đến tỉ lệ chiến thắng hay không

In [ ]:
blue_win_kill = (data_df[blue==1]['blueKills'] - data_df[blue==1]['redKills']) >0
red_win_kill = (data_df[blue==0]['redKills'] - data_df[blue==0]['blueKills']) >0
kill = (blue_win_kill .value_counts()[True] +red_win_kill.value_counts()[True])/games_count
blue_win_exp_diff = (data_df[blue==1]['blueExperienceDiff']) >0
red_win_exp_diff = (data_df[blue==0]['redExperienceDiff']) >0
ExpDiff = (blue_win_exp_diff.value_counts()[True] + red_win_exp_diff.value_counts()[True])/games_count
blue_win_gold_diff = (data_df[blue==1]['blueGoldDiff']) >0
red_win_gold_diff = (data_df[blue==0]['redGoldDiff']) >0
GoldDiff = (blue_win_gold_diff.value_counts()[True] + red_win_gold_diff.value_counts()[True])/games_count
blue_win_first_blood = data_df[blue==1]['blueFirstBlood'].value_counts()[1]
red_win_first_blood = data_df[blue==0]['redFirstBlood'].value_counts()[1]
first_blood = (blue_win_first_blood + red_win_first_blood)/games_count *100

x = ['First Blood', 'Kill', 'Experience', 'Gold']
y = [round(first_blood,3), round(kill*100,3),round(ExpDiff*100,3), round(GoldDiff*100,3)]
bars = plt.bar(x, height=y, width=.5, color='darkcyan')


plt.ylabel('Win Rate')
plt.title('So sánh tỉ lệ thắng khi hơn vàng, kinh nghiệm, mạng tiêu diệt')
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x()+0.01, yval+0.5, f"{yval}%")

plt.show()

Nhận xét: Có thể thấy cả 4 yếu tố đều ảnh hưởng đến tỉ lệ thắng nhưng tỉ lệ Experience và Gold khá vượt trội so với Kill và First Blood. Vậy có thể thấy là trong 10 phút đầu tiên thì việc có được First Blood(mạng hạ gục đầu tiên trong trận đấu) hay chênh lệch Kill có ảnh hưởng đến tỉ lệ thắng nhưng tầm quan trọng của chúng không bằng so với Experience và Gold vì vậy Experience và Gold là 2 yếu tố nên được ưu tiên hơn trong 10 phút đầu của trận đấu.

Tiếp đến sẽ là những yếu tố ít được tập trung vào hơn khi nhắc đến 1 trò chơi.

In [ ]:
blue_win_ward_placed = (data_df[blue==1]['blueWardsPlaced'] - data_df[blue==1]['redWardsPlaced']) > 0
red_win_ward_placed = (data_df[blue==0]['redWardsPlaced'] - data_df[blue==0]['blueWardsPlaced']) > 0
ward_placed = (red_win_ward_placed.value_counts()[True] + blue_win_ward_placed.value_counts()[True])/games_count

blue_win_ward_destroyed = (data_df[blue==1]['blueWardsDestroyed'] - data_df[blue==1]['redWardsDestroyed']) > 0
red_win_ward_destroyed = (data_df[blue==0]['redWardsDestroyed'] - data_df[blue==0]['blueWardsDestroyed']) > 0
ward_destroyed = (red_win_ward_destroyed.value_counts()[True] + blue_win_ward_destroyed.value_counts()[True])/games_count

blue_win_CSPerMin = (data_df[blue==1]['blueCSPerMin'] - data_df[blue==1]['redCSPerMin']) > 0
red_win_CSPerMin = (data_df[blue==0]['redCSPerMin'] - data_df[blue==0]['blueCSPerMin']) > 0
CSPerMin = (red_win_CSPerMin.value_counts()[True] + blue_win_CSPerMin.value_counts()[True])/games_count

blue_win_TotalJungleMinionsKilled = (data_df[blue==1]['blueTotalJungleMinionsKilled'] - data_df[blue==1]['redTotalJungleMinionsKilled']) > 0
red_win_TotalJungleMinionsKilled = (data_df[blue==0]['redTotalJungleMinionsKilled'] - data_df[blue==0]['blueTotalJungleMinionsKilled']) > 0
TotalJungleMinionsKilled = (red_win_TotalJungleMinionsKilled.value_counts()[True] + blue_win_TotalJungleMinionsKilled.value_counts()[True])/games_count

x = ['Ward Placed', 'Ward Destroyed', 'CSPerMin', 'Jungle Minions']
y = [round(ward_placed*100,3), round(ward_destroyed*100,3),round(CSPerMin*100,3), round(TotalJungleMinionsKilled*100,3)]
bars = plt.bar(x, height=y, width=.5, color='green')


plt.ylabel('Win Rate')
plt.title('Tỉ lệ thắng khi chênh lệch các yếu tố: Số mắt cắm, số mắt phá hủy, lính theo phút, quái rừng')
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x()+0.01, yval+0.5, f"{yval}%")



Có thể thấy là chỉ số CSPerMin và JungleMinions rất quan trọng và có ảnh hưởng nhiều đến trận đấu, chỉ số WardPlaced và WardDestroyed có ảnh hưởng nhưng có lẽ là trong 10 phút đầu của trận đấu nên ưu tiên các chỉ số kia hơn

In [ ]:
blue_win_dragon = (data_df[blue==1]['blueDragons'] - data_df[blue==1]['redDragons']) > 0
red_win_dragon = (data_df[blue==0]['redDragons'] - data_df[blue==0]['blueDragons']) > 0
red_dragon = red_win_dragon.value_counts()[True]/games_count
blue_dragon = blue_win_dragon.value_counts()[True]/games_count

blue_win_herald = (data_df[blue==1]['blueHeralds'] - data_df[blue==1]['redHeralds']) > 0
red_win_herald = (data_df[blue==0]['redHeralds'] - data_df[blue==0]['blueHeralds']) > 0
red_herald = red_win_herald.value_counts()[True]/games_count
blue_herald = blue_win_herald.value_counts()[True]/games_count

red_stat = [round(red_dragon*100,3), round(red_herald*100,3)]
blue_stat = [round(blue_dragon*100,3), round(blue_herald*100,3)]
x = ['Dragon', 'Herald']
xpos = np.arange(2)
plt.xticks(xpos, x)
bars1 = plt.bar(xpos+.2, red_stat, color='firebrick', width=.4, label="red")
for bar in bars1:
    yval = bar.get_height()
    plt.text(bar.get_x()+0.07, yval+0.25, f"{yval}%")
bars2 = plt.bar(xpos-.2, blue_stat, color='mediumblue', width=.4, label="blue")
for bar in bars2:
    yval = bar.get_height()
    plt.text(bar.get_x()+0.07, yval+0.5, f"{yval}%")
plt.legend()
plt.title("Tỉ lệ thắng khi chênh lệch Dragon và Herald ở đội xanh và đội đỏ")
plt.show()

Có 2 chỉ số khá thú vị và chênh lệch khá rõ là chỉ số Dragon và Herald và từ đó có thể thấy là khi ở đội đỏ nên ưu tiên tiêu diệt Dragon còn ở đội xanh thì tiêu diệt Herald (Có lẽ là do vị trí của Dragon và Herald trên bản đồ tác động đến việc lựa chọn tiêu diệt mục tiêu nào)

In [ ]:
categories = ['First Blood', 'Kill', 'Experience', 'Gold', 'Ward Placed', 'Ward Destroyed', 'CSPerMin', 'Jungle Minions', 'First Blood']
vehicle = [round(first_blood,3), round(kill*100,3),round(ExpDiff*100,3), round(GoldDiff*100,3), round(ward_placed*100,3), round(ward_destroyed*100,3),round(CSPerMin*100,3), round(TotalJungleMinionsKilled*100,3), round(first_blood,3)]
placement = np.linspace(start=0, stop=2*np.pi, num=len(vehicle))
plt.figure(figsize=(6,6))
plt.subplot(polar=True)
plt.plot(placement, vehicle)
lines, labels = plt.thetagrids(np.degrees(placement), labels=categories)
plt.ylim([0,100])
plt.title("So sánh tầm quan trọng của các yếu tố ảnh hưởng đến chiến thắngkhi có được sự chênh lệch trong 10 phút đầu trận đấu")
plt.show()

Kết luận: Vậy dựa vào các phân tích trên có thể trả lời câu hỏi đặt ra ban đầu là các yếu tố cần phải tập trung nhiều hơn trong 10 phút đầu để giành chiến thắng trong 1 trận đấu liên minh huyền thoại là Kill, Experience và Gold. 
Tất nhiên tất cả yếu tố trên đều quan trọng và không nên chỉ tập trung vào 3 yếu tố đó mà bỏ quên những yếu tố khác.

#### TRẦN HOÀNG NAM
##### Câu hỏi 1: Nếu chúng ta thua gold hay exp trong 10' phút đầu thì liệu chúng ta có cơ hội chiến thắng trận đấu hay không? 
Nếu trả lời được câu hỏi này, chúng ta sẽ biết được nên giữ cho khoảng cách exp và gold cách biệt ở mức nào để có khả năng lật ngược ván đấu, cũng như cách biệt như thế nào thì khả năng thắng của chúng ta chắc chắn hơn

Chúng ta sẽ xem xét sơ qua tỉ lệ thắng thua giữa 2 đội trong bộ dữ liệu xem có sự chênh lệch hay không

In [ ]:
blue_win_percent = (data_df['blueWins']==1).mean()*100
red_win_percent = 100 - blue_win_percent
plt.pie([blue_win_percent,red_win_percent],labels=['blue_win','red_win'],autopct='%1.1f%%')
plt.show()

**Nhận xét**: Có vẻ tỉ lệ thắng thua giữa 2 bên có khá là cân bằng, không có vấn đề gì nếu giải quyết câu hỏi trên tập dữ liệu này

Đầu tiên, chúng ta sẽ lọc ra những attribute của đội xanh và đỏ và lưu chúng vào 2 biến **red_att** and **blue_att**

In [ ]:
blue_att = [x for x in data_df.keys() if 'blue' in x]
blue_att.remove('blueWins')
red_att = [x for x in data_df.keys() if 'red' in x]

Chúng ta sẽ viết hàm để tính giá trị trung bình, độ lệch chuẩn của exp, gold 

In [ ]:
def mean_std(data,team='blue'):
    mean_exp = data[team+'TotalExperience'].mean()
    std_exp = data[team+'TotalExperience'].std()
    mean_gold = data[team+'TotalGold'].mean()
    std_gold = data[team+'TotalGold'].std()
    return mean_exp, std_exp, mean_gold, std_gold

Hàm std_range_percent sẽ làm các việc sau đây:
- Chuyển gold, exp về dạng z-score
- Lọc ra nhưng mẫu có z-score thuộc khoảng (std_start, std_end]
- Tính tỉ lệ thắng của các mẫu đó

In [ ]:
def std_range_percent(data,range_std_start,range_std_end,team='blue'):
    mean_exp, std_exp, mean_gold, std_gold = mean_std(data,team)
    data['z_score_'+team+'_exp'] = (data[team+'TotalExperience'] - mean_exp)/std_exp
    data['z_score_'+team+'_gold'] = (data[team+'TotalGold'] - mean_gold)/std_gold
    data_exp = data[((data['z_score_'+team+'_exp']>range_std_start)&(data['z_score_'+team+'_exp']<=range_std_end))]
    data_gold = data[(data['z_score_'+team+'_gold']>range_std_start)&(data['z_score_'+team+'_gold']<=range_std_end)]
    if team == 'blue':
        return (data_exp['blueWins']==1).mean()*100, (data_gold['blueWins']==1).mean()*100
    else:
        return (data_exp['blueWins']==0).mean()*100, (data_gold['blueWins']==0).mean()*100

Ở đây, chúng ta sẽ xem xét nếu lượng exp, gold chêch khỏi độ lệch bao nhiêu thì vẫn có khả năng gỡ lại

In [ ]:
std_check=[[-3,-2],[-2,-1],[-1,0],[0,1],[1,2],[2,3]]

In [ ]:
blue_win_percent_by_exp = []
blue_win_percent_by_gold = []
red_win_percent_by_exp = []
red_win_percent_by_gold = []
for x in std_check:
    win_exp, win_gold = std_range_percent(data_df,x[0],x[1],team='blue')
    blue_win_percent_by_exp.append(win_exp)
    blue_win_percent_by_gold.append(win_gold)
    win_exp, win_gold = std_range_percent(data_df,x[0],x[1],team='red')
    red_win_percent_by_exp.append(win_exp)
    red_win_percent_by_gold.append(win_gold)

Block này chỉ dùng để tiện cho việc rename lại index các dòng

In [ ]:
range_std_list = {}
for x in range(0,len(std_check)):
    range_std_list[x]=std_check[x]

- Chúng ta sẽ tiến hành tạo dataframe cho 4 kết quả vừa thu được ở trên, các list dữ liệu ở trên chỉ là tỉ lệ thắng, chúng ta sẽ tính cả tỉ lệ thua bằng cách lấy 100 - tỉ lệ thắng lưu vào 1 cột gọi là lose by exp, lose by gold trong các dataframe tương ứng
- Do 1 trận đấu chúng ta có 2 gồm đội xanh và đội đỏ, vì thế nên chúng ta sẽ phải tạo ra 4 dataframe gồm:
    - 

In [ ]:
#Tạo dataframe mục đích cho việc xem xét các thông số
blue_win_percent_by_exp = pd.DataFrame(blue_win_percent_by_exp)
blue_win_percent_by_exp = blue_win_percent_by_exp.rename(columns={0:'win rate calculate by exp'})
blue_win_percent_by_exp = blue_win_percent_by_exp.rename(index=range_std_list)
blue_win_percent_by_exp['lose rate calculate by exp'] = 100 - blue_win_percent_by_exp['win rate calculate by exp'] 

blue_win_percent_by_gold = pd.DataFrame(blue_win_percent_by_gold)
blue_win_percent_by_gold = blue_win_percent_by_gold.rename(columns={0:'win rate calculate by gold'})
blue_win_percent_by_gold = blue_win_percent_by_gold.rename(index=range_std_list)
blue_win_percent_by_gold['lose rate calculate by gold'] = 100 - blue_win_percent_by_gold['win rate calculate by gold'] 

red_win_percent_by_exp = pd.DataFrame(red_win_percent_by_exp)
red_win_percent_by_exp = red_win_percent_by_exp.rename(columns={0:'win rate calculate by exp'})
red_win_percent_by_exp = red_win_percent_by_exp.rename(index=range_std_list)
red_win_percent_by_exp['lose rate calculate by exp'] = 100 - red_win_percent_by_exp['win rate calculate by exp'] 

red_win_percent_by_gold = pd.DataFrame(red_win_percent_by_gold)
red_win_percent_by_gold = red_win_percent_by_gold.rename(columns={0:'win rate calculate by gold'})
red_win_percent_by_gold = red_win_percent_by_gold.rename(index=range_std_list)
red_win_percent_by_gold['lose rate calculate by gold'] = 100 - red_win_percent_by_gold['win rate calculate by gold'] 

In [ ]:
blue_win_percent_by_exp

In [ ]:
blue_win_percent_by_gold

In [ ]:
red_win_percent_by_exp

In [ ]:
red_win_percent_by_gold

Để cho dễ nhìn, chúng ta sẽ trực quan hoá hết 4 table trên bằng stackbarchart

In [ ]:
blue_win_percent_by_exp.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('blue win rate calculate by exp std')
blue_win_percent_by_gold.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.title('blue win rate calculate by gold std')
plt.xlabel('percent')
plt.show()

In [ ]:
red_win_percent_by_exp.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('red win rate calculate by exp std')
red_win_percent_by_gold.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('red win rate calculate by gold std')
plt.show()

**Nhận xét**: Nhìn chung, với sự chênh lệch về gold và exp càng cao thì tỉ lệ tách biệt thắng thua sẽ càng cao, nhưng theo biểu đồ trên, chúng ta dễ dàng nhận thấy, độ chêck trong khoản từ -1 cho đến 1 độ lệch chuẩn của gold và exp thì tỉ lệ thắng thua chỉ là khoảng 40-60 hoặc ngược lại,tính ra không quá chênh lệch với tỉ lệ thắng thua ngẫu nhiên là 50%, và nếu so tỉ lệ thắng của đội xanh,đỏ trong mẫu dữ liệu ban đầu thì cũng không chênh lệch quá nhiều, nhưng từ 1 đến 2 lại thì tỉ lệ thắng thua  lại tăng thêm hẳn 20%, nghĩa là để lật ngược 1 ván đấu, thì chúng ta cần giữ cho độ chênh về gold và exp không vượt quá 1 độ lệch chuẩn thì vẫn còn đủ khả năng để thắng ngược, tuy nhiên dù cho độ chênh lệch có cao thế nào thì vẫn không chắc chắn việc thắng thua hoàn toàn được, vẫn có khoảng 10% tỉ lệ thắng khi sự chênh lệch tầm khoảng 2 đến 3 độ lệch chuẩn, có vẻ ngoài gold và exp thì vẫn có những yếu tố khác ảnh hưởng, hoặc cũng có thể sau khi qua 10' đầu của trận đấu thì độ lệch này đã được thay đổi, chúng ta sẽ xem xét đến các mục tiêu lớn

##### Câu hỏi 2: Khi ăn các mục tiêu lớn thì khả năng thắng tăng như thế nào?

<p>Có 2 mục tiêu lớn mà chúng ta cần quan tâm là rồng và sứ giả, tạm thời chúng ta sẽ xem xét dragon trước</p>
Chúng ta sẽ lọc ra những trận đấu mà đội đỏ ăn rồng và đội xanh ăn rồng, sau đó lưu vào 2 biến <b>red_dragon</b> và <b>blue_dragon</b>

In [ ]:
red_dragon = (data_df[data_df['redDragons']!=0]).copy()
blue_dragon = (data_df[(data_df['blueDragons']!=0)]).copy()

Chúng ta sẽ tính tỉ lệ thắng đối với các đội ăn được rồng

In [ ]:
red_dragon_wins = (red_dragon["blueWins"]==0).mean()*100
blue_dragon_wins = (blue_dragon["blueWins"]==1).mean()*100
print(f'Tỉ lệ thắng của đội xanh khi ăn rồng là: {blue_dragon_wins:<.2f}%')
print(f'Tỉ lệ thắng của đội đỏ khi ăn rồng là: {red_dragon_wins:<.2f}%')

Tương tự như ở **câu hỏi 1**, chúng ta sẽ tính tỉ lệ thắng dựa trên gold và exp bằng 2 bộ dữ liệu đã lọc ra là **blue_dragon** và **red_dragon**

In [ ]:
std_check = [[-3,-2],[-2,-1],[-1,0],[0,1],[1,2],[2,3]]
blue_win_percent_by_exp = []
blue_win_percent_by_gold = []
red_win_percent_by_exp = []
red_win_percent_by_gold = []
for x in std_check:
    win_rate_by_exp_std, win_rate_by_gold_std = std_range_percent(blue_dragon,x[0],x[1])
    blue_win_percent_by_exp.append(win_rate_by_exp_std)
    blue_win_percent_by_gold.append(win_rate_by_gold_std)
    win_rate_by_exp_std, win_rate_by_gold_std = std_range_percent(red_dragon,x[0],x[1],team='red')
    red_win_percent_by_exp.append(win_rate_by_exp_std)
    red_win_percent_by_gold.append(win_rate_by_gold_std)

In [ ]:
blue_win_percent_by_exp = pd.DataFrame(blue_win_percent_by_exp)
blue_win_percent_by_exp = blue_win_percent_by_exp.rename(columns={0:'win rate calculate by exp'})
blue_win_percent_by_exp = blue_win_percent_by_exp.rename(index=range_std_list)
blue_win_percent_by_exp['lose rate calculate by exp'] = 100 - blue_win_percent_by_exp['win rate calculate by exp'] 

blue_win_percent_by_gold = pd.DataFrame(blue_win_percent_by_gold)
blue_win_percent_by_gold = blue_win_percent_by_gold.rename(columns={0:'win rate calculate by gold'})
blue_win_percent_by_gold = blue_win_percent_by_gold.rename(index=range_std_list)
blue_win_percent_by_gold['lose rate calculate by gold'] = 100 - blue_win_percent_by_gold['win rate calculate by gold'] 

red_win_percent_by_exp = pd.DataFrame(red_win_percent_by_exp)
red_win_percent_by_exp = red_win_percent_by_exp.rename(columns={0:'win rate calculate by exp'})
red_win_percent_by_exp = red_win_percent_by_exp.rename(index=range_std_list)
red_win_percent_by_exp['lose rate calculate by exp'] = 100 - red_win_percent_by_exp['win rate calculate by exp'] 

red_win_percent_by_gold = pd.DataFrame(red_win_percent_by_gold)
red_win_percent_by_gold = red_win_percent_by_gold.rename(columns={0:'win rate calculate by gold'})
red_win_percent_by_gold = red_win_percent_by_gold.rename(index=range_std_list)
red_win_percent_by_gold['lose rate calculate by gold'] = 100 - red_win_percent_by_gold['win rate calculate by gold'] 

In [ ]:
blue_win_percent_by_exp

In [ ]:
blue_win_percent_by_gold

In [ ]:
red_win_percent_by_exp

In [ ]:
red_win_percent_by_gold

In [ ]:
blue_win_percent_by_exp.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('blue win rate calculate by exp std')
blue_win_percent_by_gold.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('blue win rate calculate by gold std')
plt.show()

In [ ]:
red_win_percent_by_exp.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('red win rate calculate by exp std')
red_win_percent_by_gold.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('red win rate calculate by gold std')
plt.show()

**Nhận xét**: so sánh 4 biểu đồ ở câu 1 với 4 biểu đồ trên, chúng ta nhận thấy tỉ lệ thắng khi ăn rồng đã tăng lên đáng kể, ngay cả khi vàng và exp thua đối thủ khoảng 1 đến 2 độ lệch chuẩn thì tỉ lệ thắng vẫn hơn 30%, nếu chỉ thua trong tầm khoảng 0-1 độ lệch chuẩn thì tỉ lệ thắng lại hơn 50%, nếu không ăn rồng thì 2 con số này chỉ tầm 20% và 40% như phần biểu đồ trong **câu hỏi 1**

Tóm lại, khi đang trong thế thua của 1 ván đấu, nhất là về exp và gold thì bạn cần tập trung vào mục tiêu lớn là rồng, việc ăn rồng có thể bù đắp được việc thua về lượng exp và gold nhận được ở trong khoảng chêch lệch 1 độ lệch chuẩn

Còn 1 mục tiêu lớn còn lại là sứ giả, trong 1 trận đấu, **giả sử đội bạn đang ăn rồng, thì chúng ta nên ăn sứ giả, hay là tham gia tranh chấp rồng?**

Tương tự như phần trên, chúng ta sẽ lọc ra các trận đội xanh, đỏ ăn sứ giả và lưu vào biến **blue_herald** và **red_herald**

In [ ]:
blue_herald = (data_df[data_df['blueHeralds']!=0]).copy()
red_herald = (data_df[data_df['redHeralds']!=0]).copy()

Dễ dàng chúng ta tính được tỉ lệ thắng của các đội chọn ăn sứ giả là

In [ ]:
print(f'Tỉ lệ thắng của đội xanh khi ăn sứ giả là: {(blue_herald["blueWins"]==1).mean()*100:<.2f}')
print(f'Tỉ lệ thắng của đội đỏ khi ăn sứ giả là: {(red_herald["blueWins"]==0).mean()*100:<.2f}')

**Nhận xét**: Có vẻ như tỉ lệ thắng của các đội chọn ăn sứ giả thấp hơn là các đội chọn ăn rồng ở 10' đầu trận đấu, nhưng tỉ lệ chêch lệch chỉ khoảng 5%, chúng ta có thể tạm chấp nhận được

In [ ]:
std_check = [[-3,-2],[-2,-1],[-1,0],[0,1],[1,2],[2,3]]
blue_win_percent_by_exp = []
blue_win_percent_by_gold = []
red_win_percent_by_exp = []
red_win_percent_by_gold = []
for x in std_check:
    win_rate_by_exp_std, win_rate_by_gold_std = std_range_percent(blue_herald,x[0],x[1])
    blue_win_percent_by_exp.append(win_rate_by_exp_std)
    blue_win_percent_by_gold.append(win_rate_by_gold_std)
    win_rate_by_exp_std, win_rate_by_gold_std = std_range_percent(red_herald,x[0],x[1],team='red')
    red_win_percent_by_exp.append(win_rate_by_exp_std)
    red_win_percent_by_gold.append(win_rate_by_gold_std)

In [ ]:
blue_win_percent_by_exp = pd.DataFrame(blue_win_percent_by_exp)
blue_win_percent_by_exp = blue_win_percent_by_exp.rename(columns={0:'win rate calculate by exp'})
blue_win_percent_by_exp = blue_win_percent_by_exp.rename(index=range_std_list)
blue_win_percent_by_exp['lose rate calculate by exp'] = 100 - blue_win_percent_by_exp['win rate calculate by exp'] 

blue_win_percent_by_gold = pd.DataFrame(blue_win_percent_by_gold)
blue_win_percent_by_gold = blue_win_percent_by_gold.rename(columns={0:'win rate calculate by gold'})
blue_win_percent_by_gold = blue_win_percent_by_gold.rename(index=range_std_list)
blue_win_percent_by_gold['lose rate calculate by gold'] = 100 - blue_win_percent_by_gold['win rate calculate by gold'] 

red_win_percent_by_exp = pd.DataFrame(red_win_percent_by_exp)
red_win_percent_by_exp = red_win_percent_by_exp.rename(columns={0:'win rate calculate by exp'})
red_win_percent_by_exp = red_win_percent_by_exp.rename(index=range_std_list)
red_win_percent_by_exp['lose rate calculate by exp'] = 100 - red_win_percent_by_exp['win rate calculate by exp'] 

red_win_percent_by_gold = pd.DataFrame(red_win_percent_by_gold)
red_win_percent_by_gold = red_win_percent_by_gold.rename(columns={0:'win rate calculate by gold'})
red_win_percent_by_gold = red_win_percent_by_gold.rename(index=range_std_list)
red_win_percent_by_gold['lose rate calculate by gold'] = 100 - red_win_percent_by_gold['win rate calculate by gold'] 

In [ ]:
blue_win_percent_by_exp

In [ ]:
blue_win_percent_by_gold

In [ ]:
red_win_percent_by_exp

In [ ]:
red_win_percent_by_gold

In [ ]:
blue_win_percent_by_exp.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('blue win rate calculate by exp std')
blue_win_percent_by_gold.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('blue win rate calculate by gold std')
plt.show()

In [ ]:
red_win_percent_by_exp.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('red win rate calculate by exp std')
red_win_percent_by_gold.plot(kind='barh',stacked=True)
plt.axvline(50,color='red')
plt.ylabel('std range')
plt.xlabel('percent')
plt.title('red win rate calculate by gold std')
plt.show()

**Nhận xét**: So sánh với những đội chọn ăn rồng, thì có vẻ như tỉ lệ thắng có vẻ thấp hơn một vài %, điều này có thể chấp nhận được, nhưng chúng ta hãy để ý đến lượng gold, có vẻ nếu mà 1 đội thua quá xa về lượng gold thì việc ăn sứ giả không có bất cứ tác dụng nào cả khi so sánh với những đội chọn ăn rồng

**Kết luận rút ra sau 2 câu hỏi**: Gold và exp là 2 yếu tố quan trọng để thắng trận đấu, nhưng nếu bạn không thể nào có đủ lượng gold và exp như đối thủ, thì hãy cố gắng đừng để khoảng cách đó quá xa khỏi 1 độ lệch chuẩn, tuy nhiên, nếu không thể giữ được khoảng cách đó, thì team của bạn cần tập trung nhiều vào những mục tiêu lớn cụ thể là rồng, đặc biệt nếu team bạn đang thua về gold, thì bắt buột mục tiêu lớn phải là rồng, vì sứ giả sẽ không thể bù đắp được cho bạn nếu lượng vàng thua quá lớn